# install modules

In [ ]:
%%capture
# !pip install pytorch-lightning==1.4.9
!pip install -q pytorch-lightning wandb
!pip install gensim --upgrade
!pip install datasets transformers[sentencepiece]

In [ ]:
import numpy as np
import collections
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re,nltk,json
import random
import torch
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
SEED = 1234
from pytorch_lightning import seed_everything
seed_everything(SEED)
torch.backends.cudnn.deterministic = True
from tqdm.notebook import tqdm

Global seed set to 1234


In [ ]:
import torch.nn as nn
from pytorch_lightning import LightningModule,LightningDataModule,Trainer
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor,ModelCheckpoint,EarlyStopping,ProgressBar
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
import torch.nn.functional as F

In [ ]:
import logging
import datasets
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_metric
import transformers
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from torch.utils.data import Dataset,TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import (
    AutoModel,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    default_data_collator,
    set_seed,
    get_constant_schedule_with_warmup
)
import sys

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
logger = logging.getLogger(__name__)

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

# mount gsuit drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Creating Dataloader function

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import XLMRobertaTokenizer,BertTokenizerFast,BertTokenizer

In [ ]:
from torch.utils.data import Dataset,TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
all_checkpoints=['bert-base-multilingual-cased',
                 'sagorsarker/bangla-bert-base',
                 'neuralspace-reverie/indic-transformers-bn-bert',
                 'neuralspace-reverie/indic-transformers-bn-roberta',
                 'distilbert-base-multilingual-cased',
                 'neuralspace-reverie/indic-transformers-bn-distilbert',
                 'monsoon-nlp/bangla-electra',
                 'csebuetnlp/banglabert',
                 'neuralspace-reverie/indic-transformers-bn-xlmroberta'
                 ]

In [ ]:
tokenizer_list=[]
tokenizer_list.append('AutoTokenizer.from_pretrained(all_checkpoints[0])')
tokenizer_list.append('BertTokenizer.from_pretrained(all_checkpoints[1])')
tokenizer_list.append('BertTokenizer.from_pretrained(all_checkpoints[2])')
tokenizer_list.append('AutoTokenizer.from_pretrained(all_checkpoints[3])')
tokenizer_list.append('AutoTokenizer.from_pretrained(all_checkpoints[4])')
tokenizer_list.append('AutoTokenizer.from_pretrained(all_checkpoints[5])')
tokenizer_list.append('AutoTokenizer.from_pretrained(all_checkpoints[6])')
tokenizer_list.append('AutoTokenizer.from_pretrained(all_checkpoints[7])')
tokenizer_list.append('XLMRobertaTokenizer.from_pretrained(all_checkpoints[8])')

In [ ]:
%cd /content
!git clone https://github.com/crux82/ganbert

/content
Cloning into 'ganbert'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 77 (delta 33), reused 54 (delta 18), pack-reused 0
Unpacking objects: 100% (77/77), done.


In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

##  the dataloader class

In [ ]:
traindf = pd.read_csv('/content/drive/MyDrive/Thesis BRAC/trainLatest.csv')
testdf = pd.read_csv('/content/drive/MyDrive/Thesis BRAC/testLatest.csv')
unlabelled_df=pd.read_csv('/content/drive/MyDrive/Thesis BRAC/combined_mini.csv')
traindf.shape,unlabelled_df.shape,testdf.shape

((12044, 7), (65406, 2), (3012, 7))

In [ ]:
import wandb
!wandb login
run = wandb.init()
x_values = [1,2,3,4]
y_values = [1,2,3,4]
data = [[x, y] for (x, y) in zip(x_values, y_values)]
table = wandb.Table(data=data, columns = ["x", "y"])
wandb.log({"my_custom_plot_id" : wandb.plot.line(table, "x", "y",
           title="Custom Y vs X Line Plot")})

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: colab-team (use `wandb login --relogin` to force relogin)


In [ ]:
class MNISTDataModule(LightningDataModule):
  def __init__(
      self,
      batch_size: int = 64,
      num_workers: int = 4,
      label_list=[0,1,2],
      model_number = 0,
      max_seq_length=64,
      choose_size = -1
  ):
      super().__init__()
      self.batch_size = batch_size
      self.num_workers = num_workers  
      self.label_list = label_list
      self.num_classes = len(self.label_list)
      self.model_name =all_checkpoints[model_number]
      self.max_seq_length = max_seq_length
      self.tokenizer = eval(tokenizer_list[model_number])
      self.traindf = pd.read_csv('/content/drive/MyDrive/Thesis BRAC/trainLatest.csv')
      self.traindf = self.traindf.sample(frac=1,random_state=42).reset_index(drop=True)
      self.testdf = pd.read_csv('/content/drive/MyDrive/Thesis BRAC/testLatest.csv')
      self.testdf= self.testdf.sample(frac=1,random_state=42).reset_index(drop=True)
      self.label_map = {}
      for (i, label) in enumerate(self.label_list):
        self.label_map[label] = i
        # Assign test dataset for use in dataloader(s)
      
      #The labeled (test) dataset is assigned with a mask set to True
      self.test_examples = list(zip(self.testdf['cleaned_title'].values,self.testdf['label'].values))
      self.test_label_masks = np.ones(len(self.test_examples), dtype=bool)
      #------------------------------
      #   Load the train dataset
      #------------------------------
      sizes = [0.05,0.1,0.3,0.2,0.4,0.6,0.8,1]
      labelled_size = int(sizes[choose_size]*self.traindf['cleaned_title'].shape[0])
      # self.train_examples = list(zip(self.traindf['cleaned_title'].values,self.traindf['label'].values))
      self.train_examples = list(zip(self.traindf['cleaned_title'].values[:labelled_size],self.traindf['label'].values[:labelled_size]))
      
      
  def generate_data_loader(self,input_examples,label_map, do_shuffle = False):
    '''
    Generate a Dataloader given the input examples, eventually masked if they are 
    to be considered NOT labeled.
    '''
    #-----------------------------------------------
    # Generate input examples to the Transformer
    #-----------------------------------------------
    input_ids = []
    input_mask_array = []
    label_id_array = []

    # Tokenization 
    for text,label in input_examples:
      encoded_sent = self.tokenizer.encode(text, add_special_tokens=True, max_length=self.max_seq_length, padding="max_length", truncation=True)
      input_ids.append(encoded_sent)
      label_id_array.append(label_map[label])
      
    
    # Attention to token (to ignore padded input wordpieces)
    for sent in input_ids:
      att_mask = [int(token_id > 0) for token_id in sent]                          
      input_mask_array.append(att_mask)
    # Convertion to Tensor
    input_ids = torch.tensor(input_ids) 
    input_mask_array = torch.tensor(input_mask_array)
    label_id_array = torch.tensor(label_id_array, dtype=torch.long)

    # Building the TensorDataset
    dataset = TensorDataset(input_ids, input_mask_array, label_id_array)
    # return dataset
    if do_shuffle:
      sampler = RandomSampler
    else:
      sampler = SequentialSampler

    # Building the DataLoader
    return DataLoader(
                dataset,  # The training samples.
                sampler = sampler(dataset), 
                batch_size = self.batch_size,
                pin_memory = True,
                num_workers=self.num_workers) # Trains with this batch size.

  def train_dataloader(self):
      return self.generate_data_loader(self.train_examples,\
                                  self.label_map, do_shuffle = True)

  def val_dataloader(self):
      return self.generate_data_loader(self.test_examples, self.label_map,\
                                  do_shuffle = False)

  # def test_dataloader(self):
  #     return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=self.num_workers)

In [ ]:
gb_dataset = MNISTDataModule(model_number =0)
len(gb_dataset.train_examples)

12044

In [ ]:
for batch in gb_dataset.train_dataloader():
  print(batch)
  print(batch[0].shape,batch[1].shape,batch[2].shape)
  break

[tensor([[  101,   963, 11199,  ...,     0,     0,     0],
        [  101, 58169,   100,  ...,     0,     0,     0],
        [  101, 24288,  1001,  ...,     0,     0,     0],
        ...,
        [  101,   941, 20725,  ...,     0,     0,     0],
        [  101,   978, 29806,  ...,     0,     0,     0],
        [  101,   970, 37422,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
        1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0])]
torch.Size([64, 64]) torch.Size([64, 64]) torch.Size([64])


In [ ]:
for batch in gb_dataset.val_dataloader():
  print(batch[0].shape,batch[1].shape,batch[2].shape)
  break

torch.Size([64, 64]) torch.Size([64, 64]) torch.Size([64])


In [ ]:
class TransformerForSequenceClassification(nn.Module):
    """
    Simplified version of the same class by HuggingFace.
    See transformers/modeling_distilbert.py in the transformers repository.
    """

    def __init__(
        self, pretrained_model_name: str, num_classes: int = 2, dropout: float = 0.5,
        mean_pool: bool=True
    ):
        """
        Args:
            pretrained_model_name (str): HuggingFace model name.
                See transformers/modeling_auto.py
            num_classes (int): the number of class labels
                in the classification task
        """
        super().__init__()
        config = AutoConfig.from_pretrained(
            pretrained_model_name, num_labels=num_classes
        )
        for k in config.to_dict().keys():
          if 'dropout' in k and 'classifier' not in k:
            config.update({k:0.3})

        self.model = AutoModel.from_pretrained(pretrained_model_name, config=config)
        # self.model.encoder = reinit_autoencoder_model(self.model.encoder, reinit_num_layers=1)
        # print('I am here in init method')
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.dropout = nn.Dropout(dropout)
        self.mean_pool=mean_pool
        self.num_labels = num_classes
        if torch.cuda.is_available():
          self.model.cuda()
          self.classifier.cuda()
          self.dropout.cuda()
    
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        output_attentions=None,
        output_hidden_states=None,
        **kwargs
    ):
        # print('I am in forward method')
        output = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
        # print('I got output')
        hidden_state = output[0]  # (bs, seq_len, dim)
        outputs = (hidden_state,)
        if not self.mean_pool:
          pooled_output = hidden_state[:, 0]  # (bs, dim)
        else:
          pooled_output = hidden_state.mean(axis=1)  # (bs, dim)

        # pooled_output = self.pre_classifier(pooled_output)  # (bs, dim)
        # pooled_output = nn.ReLU()(pooled_output)  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        # print('I got output2')
        # print(pooled_output.shape)
        logits = self.classifier(pooled_output)  # (bs, dim)
        # print('I got output3')
        # outputs = (logits,) + output[1:]
        outputs = (logits,) + outputs
        # print('labels:',labels)
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from collections import OrderedDict
def compute_metrics(pred):
    preds,labels = pred
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    # print(acc,precision,recall,f1)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
    # return {'accuracy':acc}

In [ ]:
def build_optimizer(model, config):
    transformer_vars = [i for i in model.parameters()]
    optimizer = torch.optim.AdamW(transformer_vars, lr=config.learning_rate)
    num_train_steps = int(config.num_train_examples / config.batch_size * config.epochs)
    num_warmup_steps = int(num_train_steps * config.warmup_proportion)

    scheduler = get_constant_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = num_warmup_steps)
      
    return optimizer,scheduler


# manual training

In [ ]:
all_checkpoints=['bert-base-multilingual-cased',
                 'sagorsarker/bangla-bert-base',
                 'neuralspace-reverie/indic-transformers-bn-bert',
                 'neuralspace-reverie/indic-transformers-bn-roberta',
                 'distilbert-base-multilingual-cased',
                 'neuralspace-reverie/indic-transformers-bn-distilbert',
                 'monsoon-nlp/bangla-electra',
                 'csebuetnlp/banglabert',
                 'neuralspace-reverie/indic-transformers-bn-xlmroberta'
                 ]

In [ ]:
import math
sweep_config = {
    'learning_rate': 1e-05,
      'batch_size':64,
      'warmup_proportion':0.1,
      'num_train_examples': len(gb_dataset.train_examples),
      'model_number': 7,
      'epochs': 5,
     'device': 'cuda'}
sweep_config

{'batch_size': 64,
 'device': 'cuda',
 'epochs': 5,
 'learning_rate': 1e-05,
 'model_number': 7,
 'num_train_examples': 12044,
 'warmup_proportion': 0.1}

## training methods

In [ ]:
import collections
def train_epoch(model, dataset, optimizers,config,epoch):
  training_step_outputs=[]
  # Measure how long the training epoch takes.
  t0 = time.time()
  print_each_n_step = 100
  optimizer = optimizers[0]
  scheduler= optimizers[1]
  train_dl = dataset.train_dataloader()
  test_dl = dataset.val_dataloader()
  n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)
  # Put the model into training mode.
  model.train()
  # For each batch of training data...
  
  for step, batch in enumerate(train_dl):
    # Progress update every print_each_n_step batches.
    if step % print_each_n_step == 0 and not step == 0:
        # Calculate elapsed time in minutes.
        elapsed = format_time(time.time() - t0)    
        # Report progress.
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dl), elapsed))

    # Unpack this training batch from our dataloader. 
    b_input_ids = batch[0].to(config.device)
    b_input_mask = batch[1].to(config.device)
    b_labels = batch[2].to(config.device)
    real_batch_size=b_input_ids.shape[0]
    loss,logits,hidden_states = model(input_ids=b_input_ids,attention_mask=b_input_mask,labels=b_labels)
    preds = torch.argmax(logits,dim=1)
    #---------------------------------
    #  OPTIMIZATION
    #---------------------------------
    # Avoid gradient accumulation
    optimizer.zero_grad()

    # Calculate weigth updates
    # retain_graph=True is required since the underlying graph will be deleted after backward
    loss.backward() 
    
    #clip grad norm
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Apply modifications
    optimizer.step()
    scheduler.step()
    metrics = {'train/loss_step':loss}
    if step + 1 < n_steps_per_epoch:
      wandb.log(metrics)
    
    training_step_outputs.append(OrderedDict({'loss':loss.detach().cpu(),\
                                              'preds':preds.detach().cpu(),\
                                              'labels':b_labels.detach().cpu(),
                                              }))
    
  
  all_losses = np.mean([x['loss'].item() for x in training_step_outputs])
  all_preds = [y.item() for x in training_step_outputs for y in x['preds']]
  all_labels =  [y.item() for x in training_step_outputs for y in x['labels']]
  result = compute_metrics((all_preds,all_labels))

  epoch_metrics = {"train/epoch_accuracy": result['accuracy'],
                   "train/epoch_precision_nc": result['precision'][0],
                   "train/epoch_recall_nc":result['recall'][0],
                   "train/epoch_f1_nc": result['f1'][0],
                   "train/epoch_precision": result['precision'][1],
                   "train/epoch_recall": result['recall'][1],
                   "train/epoch_f1": result['f1'][1],
                   'train/epoch_loss':all_losses,
                   'epoch':epoch
                   }
  wandb.log(epoch_metrics) 
  print('Training acc',result['accuracy'])
  print('Training prec non-clickbait',result['precision'][0])
  print('Training recall non-clickbait',result['recall'][0])
  print('Training f1 non-clickbait',result['f1'][0])
  print('Training prec clickbait',result['precision'][1])
  print('Training recall clickbait',result['recall'][1])
  print('Training f1 clickbait',result['f1'][1])
  print('Training epoch loss',all_losses)

  training_time = format_time(time.time() - t0)                    
  print("  Training epcoh took: {:}".format(training_time))
  validation_metrics = validate_model(model,test_dl,config,epoch)
  return validation_metrics

In [ ]:
def validate_model(model, test_dl,config,epoch):
  print("Running Test...")
  t0 = time.time()
  val_step_outputs = []
  nll_loss = torch.nn.CrossEntropyLoss()
  # Put the model in evaluation mode--the dropout layers behave differently
  # during evaluation.
  model.eval()

  for batch in test_dl:
    # Unpack this training batch from our dataloader. 
    b_input_ids = batch[0].to(config.device)
    b_input_mask = batch[1].to(config.device)
    b_labels = batch[2].to(config.device)
    # print('taken input from batch')
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        
      loss,logits,hidden_states = model(input_ids=b_input_ids,attention_mask=b_input_mask,labels=b_labels)
    
    # Accumulate the test loss.
    # print(filtered_logits.shape)
    # print('run discriminator')
    preds = torch.argmax(logits, dim=1)
    loss = nll_loss(logits, b_labels)
    # print('calculated everythin')
    wandb.log({'val/loss':loss})
    # print('logged everythin')

    val_step_outputs.append(OrderedDict({'loss':loss.detach().cpu(),\
                                          'preds':preds.detach().cpu(),'labels':b_labels.detach().cpu()}))
  
  all_losses = np.mean([x['loss'].item() for x in val_step_outputs])
  
  all_preds = [y.item() for x in val_step_outputs for y in x['preds']]
  # print(all_preds)
  all_labels =  [y.item() for x in val_step_outputs for y in x['labels']]
  # print(all_labels)
  result = compute_metrics((all_preds,all_labels))
  epoch_metrics = {"val/epoch_accuracy": result['accuracy'],
                   "val/epoch_precision_nc": result['precision'][0],
                   "val/epoch_recall_nc":result['recall'][0],
                   "val/epoch_f1_nc": result['f1'][0],
                   "val/epoch_precision": result['precision'][1],
                   "val/epoch_recall": result['recall'][1],
                   "val/epoch_f1": result['f1'][1],
                   'val/epoch_loss':all_losses,
                   'epoch':epoch}
  wandb.log(epoch_metrics)
  print('Val acc',result['accuracy'])
  print('Val prec clickbait',result['precision'][1])
  print('Val recall clickbait',result['recall'][1])
  print('Val f1 clickbait',result['f1'][1])
  print('Val prec non clickbait',result['precision'][0])
  print('Val recall non clickbait',result['recall'][0])
  print('Val f1 non clickbait',result['f1'][0])
  print('Val loss',all_losses)
  test_time = format_time(time.time() - t0)
  print("  Test took: {:}".format(test_time))
  return epoch_metrics
  # return val_step_outputs

## train

In [ ]:
import wandb
!wandb login

wandb: Currently logged in as: colab-team (use `wandb login --relogin` to force relogin)


In [ ]:
# !rm -r /content/wandb
# !rm -r /content/lightning_logs
!rm -r /content/clickbait_title

In [ ]:
!mkdir /content/clickbait_title
!chmod 765 /content/clickbait_title

In [ ]:
best_test_f1 =0 
seed_everything(42)
run = wandb.init(project="banbait",entity='colab-team',config=sweep_config)
run = wandb.init(project="banbait_comparison",entity='colab-team',config=sweep_config)
# run = wandb.init(config=sweep_config)
config = wandb.config
main_dataset = MNISTDataModule(model_number = config.model_number)
clickbaitmodel = TransformerForSequenceClassification(
    pretrained_model_name = all_checkpoints[config.model_number])
wandb.watch(clickbaitmodel, log="gradients")
optimizers = build_optimizer(clickbaitmodel, config)

Global seed set to 42


In [ ]:
for epoch in tqdm(range(config.epochs)):
    validation_metrics = train_epoch(clickbaitmodel,main_dataset, optimizers,config,epoch+1)
    model_name = all_checkpoints[config.model_number].split('/')[-1]
    test_f1 = validation_metrics['val/epoch_f1']
    save_best_model = False
    if epoch==0:
      best_test_f1 = test_f1
      save_best_model = True
      
    elif test_f1>best_test_f1:
      best_test_f1 = test_f1
      save_best_model = True

    if save_best_model:
      PATH=f'/content/clickbait_title/{model_name}.pt'
      torch.save({
            'epoch': epoch,
            'model': clickbaitmodel.state_dict(),
            'optimizer_state_dict': optimizers[0].state_dict(),
            'test_loss': validation_metrics['val/epoch_loss'],
            }, PATH)
      save_best_model = False
run.summary['highest_f1_score'] = best_test_f1
model_artifact = wandb.Artifact(
      f"{model_name}_{best_test_f1:.3f}", type=f'{model_name}',
      description=f"{model_name}-test f1 clickbait:{best_test_f1}",
      metadata=dict(config))
PATH=f'/content/clickbait_title/{model_name}.pt'
model_artifact.add_file(PATH)
run.log_artifact(model_artifact)

  0%|          | 0/10 [00:00<?, ?it/s]

  Batch   100  of    189.    Elapsed: 0:00:41.
Training acc 0.8356027897708402
Training prec non-clickbait 0.8605904691902487
Training recall non-clickbait 0.8929706368374222
Training f1 non-clickbait 0.8764815970056145
Training prec clickbait 0.7830456068023706
Training recall clickbait 0.7275556619583433
Training f1 clickbait 0.7542814594192107
Training epoch loss 0.36714833194301244
  Training epcoh took: 0:01:16
Running Test...
Val acc 0.7901726427622842
Val prec clickbait 0.6970534069981584
Val recall clickbait 0.7141509433962264
Val f1 clickbait 0.7054986020503262
Val prec non clickbait 0.8426791277258567
Val recall non clickbait 0.8314549180327869
Val f1 non clickbait 0.8370293965961837
Val loss 0.487850371748209
  Test took: 0:00:06
  Batch   100  of    189.    Elapsed: 0:00:42.
Training acc 0.8419129857190302
Training prec non-clickbait 0.8635532252164371
Training recall non-clickbait 0.9002160925384518
Training f1 non-clickbait 0.8815036096589495
Training prec clickbait 0.795

In [ ]:
for i in range(1,3):
  sizes = [0.05,0.1,0.3,0.2,0.4,0.6,0.8,1]
  sweep_config['num_train_examples'] = int(sizes[i]*len(gb_dataset.train_examples))
  best_test_f1 =0 
  seed_everything(42)
  # run = wandb.init(project="banbait",entity='colab-team',config=sweep_config)
  run = wandb.init(project="banbait_comparison",entity='colab-team',config=sweep_config)
  # run = wandb.init(config=sweep_config)
  config = wandb.config
  main_dataset = MNISTDataModule(model_number = config.model_number,choose_size = i)
  print(len(main_dataset.train_examples))
  clickbaitmodel = TransformerForSequenceClassification(
      pretrained_model_name = all_checkpoints[config.model_number])
  # wandb.watch(clickbaitmodel, log="gradients")
  optimizers = build_optimizer(clickbaitmodel, config)
  for epoch in tqdm(range(config.epochs)):
      validation_metrics = train_epoch(clickbaitmodel,main_dataset, optimizers,config,epoch+1)
      model_name = all_checkpoints[config.model_number].split('/')[-1]
      test_f1 = validation_metrics['val/epoch_f1']
      save_best_model = False
      if epoch==0:
        best_test_f1 = test_f1
        save_best_model = True
        
      elif test_f1>best_test_f1:
        best_test_f1 = test_f1
        save_best_model = True

      # if save_best_model:
      #   PATH=f'/content/clickbait_title/{model_name}.pt'
      #   torch.save({
      #         'epoch': epoch,
      #         'model': clickbaitmodel.state_dict(),
      #         'optimizer_state_dict': optimizers[0].state_dict(),
      #         'test_loss': validation_metrics['val/epoch_loss'],
      #         }, PATH)
        save_best_model = False
  run.summary['highest_f1_score'] = best_test_f1
  wandb.log({'f1_score':best_test_f1,'size':sizes[i]})
  wandb.finish()

Global seed set to 42


1204


Some weights of the model checkpoint at csebuetnlp/banglabert were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/5 [00:00<?, ?it/s]

Training acc 0.659468438538206
Training prec non-clickbait 0.6632911392405063
Training recall non-clickbait 0.986198243412798
Training f1 non-clickbait 0.79313824419778
Training prec clickbait 0.42105263157894735
Training recall clickbait 0.019656019656019656
Training f1 clickbait 0.03755868544600939
Training epoch loss 0.6513169188248483
  Training epcoh took: 0:00:14
Running Test...


...

  0%|          | 0/5 [00:00<?, ?it/s]

Training acc 0.6473844450595073
Training prec non-clickbait 0.6527503526093089
Training recall non-clickbait 0.9817564700890963
Training f1 non-clickbait 0.7841409691629956
Training prec clickbait 0.36764705882352944
Training recall clickbait 0.019904458598726114
Training f1 clickbait 0.0377643504531722
Training epoch loss 0.6516887022737872
  Training epcoh took: 0:00:44
Running Test...


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Val acc 0.648074369189907
Val prec clickbait 0.0
Val recall clickbait 0.0
Val f1 clickbait 0.0
Val prec non clickbait 0.648074369189907
Val recall non clickbait 1.0
Val f1 non clickbait 0.7864625302175665
Val loss 0.6205444596707821
  Test took: 0:00:13
Training acc 0.6662053694990313
Training prec non-clickbait 0.6652311225954637
Training recall non-clickbait 0.9830292745014849
Training f1 non-clickbait 0.7934931506849315
Training prec clickbait 0.6923076923076923
Training recall clickbait 0.07165605095541401
Training f1 clickbait 0.12987012987012986
Training epoch loss 0.612248490776932
  Training epcoh took: 0:00:43
Running Test...
Val acc 0.7187915006640107
Val prec clickbait 0.6297198538367844
Val recall clickbait 0.4877358490566038
Val f1 clickbait 0.5497076023391813
Val prec non clickbait 0.7521679598356915
Val recall non clickbait 0.8442622950819673
Val f1 non clickbait 0.7955587738353851
Val loss 0.5721602483342091
  Test took: 0:00:13
Training acc 0.7431497370606145
Training 

epoch,▁▁▃▃▅▅▆▆██
f1_score,▁
size,▁
train/epoch_accuracy,▁▂▆▇█
train/epoch_f1,▁▂▇▇█
train/epoch_f1_nc,▁▂▅▆█
train/epoch_loss,█▇▄▂▁
train/epoch_precision,▁▇▇██
train/epoch_precision_nc,▁▂▆▇█
train/epoch_recall,▁▂▆▇█
train/epoch_recall_nc,██▂▁▁


In [ ]:
for i in range(2,len(all_checkpoints)):
  sweep_config['model_number'] = i
  best_test_f1 =0 
  seed_everything(42)
  run = wandb.init(project="banbait",entity='colab-team',config=sweep_config)
  # run = wandb.init(config=sweep_config)
  config = wandb.config
  main_dataset = MNISTDataModule(model_number = config.model_number)
  clickbaitmodel = TransformerForSequenceClassification(
      pretrained_model_name = all_checkpoints[config.model_number])
  wandb.watch(clickbaitmodel, log="gradients")
  optimizers = build_optimizer(clickbaitmodel, config)
  for epoch in tqdm(range(config.epochs)):
    validation_metrics = train_epoch(clickbaitmodel,main_dataset, optimizers,config,epoch+1)
    model_name = all_checkpoints[config.model_number].split('/')[-1]
    test_f1 = validation_metrics['val/epoch_f1']
    save_best_model = False
    if epoch==0:
      best_test_f1 = test_f1
      save_best_model = True
      
    elif test_f1>best_test_f1:
      best_test_f1 = test_f1
      save_best_model = True

    if save_best_model:
      PATH=f'/content/clickbait_title/{model_name}.pt'
      torch.save({
            'epoch': epoch,
            'model': clickbaitmodel.state_dict(),
            'optimizer_state_dict': optimizers[0].state_dict(),
            'test_loss': validation_metrics['val/epoch_loss'],
            }, PATH)
      save_best_model = False
  run.summary['highest_f1_score'] = best_test_f1
  model_artifact = wandb.Artifact(
        f"{model_name}_{best_test_f1:.3f}", type=f'{model_name}',
        description=f"{model_name}-test f1 clickbait:{best_test_f1}",
        metadata=dict(config))
  PATH=f'/content/clickbait_title/{model_name}.pt'
  model_artifact.add_file(PATH)
  run.log_artifact(model_artifact)
  wandb.finish()

Global seed set to 42
wandb: Currently logged in as: colab-team (use `wandb login --relogin` to force relogin)


Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/488 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/516M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralspace-reverie/indic-transformers-bn-bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Downloading:   0%|          | 0.00/516M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralspace-reverie/indic-transformers-bn-xlmroberta were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/10 [00:00<?, ?it/s]

  Batch   100  of    189.    Elapsed: 0:00:32.
Training acc 0.6741946197276653
Training prec non-clickbait 0.704533665317979
Training recall non-clickbait 0.8632261344858269
Training f1 non-clickbait 0.7758482805895122
Training prec clickbait 0.5525987525987526
Training recall clickbait 0.31817093607852526
Training f1 clickbait 0.40382862351868737
Training epoch loss 0.6044193083647067
  Training epcoh took: 0:00:56
Running Test...
Val acc 0.7652722443559097
Val prec clickbait 0.7517831669044223
Val recall clickbait 0.4971698113207547
Val f1 clickbait 0.5985235661555934
Val prec non clickbait 0.7693639117265253
Val recall non clickbait 0.9108606557377049
Val f1 non clickbait 0.8341543513957306
Val loss 0.5033654030412436
  Test took: 0:00:04
  Batch   100  of    189.    Elapsed: 0:00:32.
Training acc 0.7766522749916971
Training prec non-clickbait 0.8092223151355872
Training recall non-clickbait 0.8610652091013092
Training f1 non-clickbait 0.8343392043355093
Training prec clickbait 0.70

epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
train/epoch_accuracy,▁▄▅▅▆▆▇▇██
train/epoch_f1,▁▅▅▆▆▇▇▇██
train/epoch_f1_nc,▁▄▄▅▆▆▇▇██
train/epoch_loss,█▆▅▄▄▃▂▂▁▁
train/epoch_precision,▁▄▅▅▆▆▇▇██
train/epoch_precision_nc,▁▄▅▅▆▇▇▇██
train/epoch_recall,▁▅▅▆▆▇▇▇██
train/epoch_recall_nc,▁▁▂▃▄▅▅▆▇█
train/loss_step,██▇▆▅▇▄▄▅▃▅▅▅▄▄▅▄▃▄▄▃▅▄▃▂▄▃▂▂▅▄▃▂▁▂▂▂▂▃▁
val/epoch_accuracy,▁▅▅▅▆▇▇█▇█


In [ ]:
wandb.finish()